In [1]:
import fitbit as fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import numpy as np
import datetime
from datetime import timedelta  

from src import secrets as secret

In [4]:
#! pip show oauthlib

Name: oauthlib
Version: 2.1.0
Summary: A generic, spec-compliant, thorough implementation of the OAuth request-signing logic
Home-page: https://github.com/oauthlib/oauthlib
Author: Idan Gazit
Author-email: idan@gazit.me
License: BSD
Location: c:\users\arulc\anaconda3\lib\site-packages
Requires: 
Required-by: requests-oauthlib


In [4]:
#! pip show requests-oauthlib

Name: requests-oauthlib
Version: 1.1.0
Summary: OAuthlib authentication support for Requests.
Home-page: https://github.com/requests/requests-oauthlib
Author: Kenneth Reitz
Author-email: me@kennethreitz.com
License: ISC
Location: c:\users\arulc\anaconda3\lib\site-packages
Requires: requests, oauthlib
Required-by: google-auth-oauthlib


## Client Authentication

Create API clients as follows:
* auth2_client - for V1 of API
* [auth2_client_new](https://github.com/orcasgit/python-fitbit/issues/128) - for V1.2 of API (required for Sleep levels)

In [7]:
CLIENT_ID = secret.CLIENT_ID
CLIENT_SECRET = secret.CLIENT_SECRET
DB_USER = secret.DB_USER
DB_PASSWD = secret.DB_PASSWD
DB_HOST = secret.DB_HOST

In [8]:
#https://dev.fitbit.com/apps/details/22CZJ9
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()

ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])

##Build Auth Clients
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

#NEed this seeparately because version 1 doesn't give sleep breakdown
auth2_client_new = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)
auth2_client_new.API_VERSION = 1.2

[28/Jan/2022:18:00:23] ENGINE Listening for SIGTERM.
[28/Jan/2022:18:00:23] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[28/Jan/2022:18:00:23] ENGINE Set handler for console events.
[28/Jan/2022:18:00:23] ENGINE Started monitor thread 'Autoreloader'.
[28/Jan/2022:18:00:24] ENGINE Serving on http://127.0.0.1:8080
[28/Jan/2022:18:00:24] ENGINE Bus STARTED


127.0.0.1 - - [28/Jan/2022:18:00:26] "GET /?code=e86d1d6918b8400cbfb0d61ecb995524f615cf46&state=uXs5TmAtqw7DC0ywzY3SSnf44T56ei HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 Edg/97.0.1072.69"


[28/Jan/2022:18:00:27] ENGINE Bus STOPPING
[28/Jan/2022:18:00:27] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[28/Jan/2022:18:00:27] ENGINE Removed handler for console events.
[28/Jan/2022:18:00:27] ENGINE Stopped thread 'Autoreloader'.
[28/Jan/2022:18:00:27] ENGINE Bus STOPPED
[28/Jan/2022:18:00:27] ENGINE Bus EXITING
[28/Jan/2022:18:00:27] ENGINE Waiting for child threads to terminate...
[28/Jan/2022:18:00:27] ENGINE Bus EXITED
[28/Jan/2022:18:00:27] ENGINE Waiting for thread Thread-19.


## Database layer

In [12]:
#!pip install mysql-connector-python

  Using cached mysql_connector_python-8.0.28-cp38-cp38-win_amd64.whl (7.2 MB)


In [11]:
#!pip uninstall mysql-connector --yes

Found existing installation: mysql-connector 2.2.9
Uninstalling mysql-connector-2.2.9:
  Successfully uninstalled mysql-connector-2.2.9


In [8]:
# import mysql.connector

# cnx = mysql.connector.connect(user='root', password='Sumai1657',
#                               host='127.0.0.1',
#                               database='fitbitdb',
#                               auth_plugin='mysql_native_password')

# try:
#    cursor = cnx.cursor()
#    cursor.execute("""
#       select max(date) from fitbit_logs
#    """)
#    result = cursor.fetchall()
#    print(result)
# finally:
#     cnx.close()

[(datetime.date(2021, 12, 20),)]


In [9]:
# DATA FROM MySQL AS DATABASE
import mysql.connector

DATE_FORMAT = '%Y-%m-%d'
databaseName = 'fitbitdb'

def connectAndLoadDb():
    print("Connecting database...")
    cnx = mysql.connector.connect(user=DB_USER, password=DB_PASSWD,
                              host=DB_HOST,
                              database=databaseName,
                              auth_plugin='mysql_native_password')
    print("Database connected!")
    return cnx;

def getLastEntryDate(database):
#     lastDateEntry = database.iloc[-1]['Date']
#     lastDateEntry = datetime.datetime.strptime(lastDateEntry, DATE_FORMAT)   
    cursor = database.cursor()
    cursor.execute("""
      select max(date) from fitbit_logs
    """)
    lastDateEntry = cursor.fetchall()
    return lastDateEntry[0][0]

def addEntriesInDB(dictionary, database):
    #print(dictionary)
    database = database.append(dictionary, ignore_index=True)
    return addEntriesInDB;

def writeDbToExcelFile(database):
    print('Writing database to filename: '+ databaseFilename)
    writer = pd.ExcelWriter(databaseFilename)
    database.to_excel(writer, 'main')
    writer.save()
    print('Database updated with new entries!!')
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);    

In [54]:
database = connectAndLoadDb()

lastEntryDate = getLastEntryDate(database)

Connecting database...
Database connected!


In [57]:
lastEntryDate

[(datetime.date(2021, 12, 20),)]

In [56]:
lastDateEntry = prettyPrintDate(lastEntryDate[0][0])

In [58]:
lastEntryDate = datetime.datetime.strptime(lastDateEntry, DATE_FORMAT)

In [59]:
dateToFetch = lastEntryDate + timedelta(days=1)

In [ ]:
prettyPrintDate()

In [25]:
activitiyResponse = auth2_client.activities(date='2021-12-25')
print(activitiyResponse)

{'activities': [{'activityId': 90013, 'activityParentId': 90013, 'activityParentName': 'Walk', 'calories': 77, 'description': 'Walking less than 2 mph, strolling very slowly', 'duration': 1076000, 'hasActiveZoneMinutes': True, 'hasStartTime': True, 'isFavorite': False, 'lastModified': '2021-12-26T00:29:28.000Z', 'logId': 44995049775, 'name': 'Walk', 'startDate': '2021-12-25', 'startTime': '18:07', 'steps': 1700}], 'goals': {'activeMinutes': 30, 'caloriesOut': 2048, 'distance': 5, 'floors': 10, 'steps': 4000}, 'summary': {'activeScore': -1, 'activityCalories': 357, 'caloriesBMR': 1172, 'caloriesOut': 1459, 'distances': [{'activity': 'total', 'distance': 1.6}, {'activity': 'tracker', 'distance': 1.6}, {'activity': 'loggedActivities', 'distance': 0}, {'activity': 'veryActive', 'distance': 0.17}, {'activity': 'moderatelyActive', 'distance': 0.51}, {'activity': 'lightlyActive', 'distance': 0.91}, {'activity': 'sedentaryActive', 'distance': 0}], 'elevation': 20, 'fairlyActiveMinutes': 13, 'f

In [9]:
# DATA FROM EXCEL AS DATABASE
DATE_FORMAT = '%Y-%m-%d'
databaseFilename = 'data/database_main.xls'

def connectAndLoadDb():
    print("Connecting database...")
    database = pd.read_excel(databaseFilename)
    print("Database connected!")
    return database;

def getLastEntryDate(database):
    lastDateEntry = database.iloc[-1]['Date']
    lastDateEntry = datetime.datetime.strptime(lastDateEntry, DATE_FORMAT)    
    return lastDateEntry

def addEntriesInDB(dictionary, database):
    #print(dictionary)
    database = database.append(dictionary, ignore_index=True)
    return addEntriesInDB;

def writeDbToExcelFile(database):
    print('Writing database to filename: '+ databaseFilename)
    writer = pd.ExcelWriter(databaseFilename)
    database.to_excel(writer, 'main')
    writer.save()
    print('Database updated with new entries!!')
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);    

## Activity and Sleep Parsers

In [10]:
def safeGet(obj, key, defaultVal = np.nan):
    return obj.get(key, defaultVal)  

def percent(val):
    return np.ceil(val*100)

def filterArrObj(arrList, keyName, keyValue):
    for arrItem in arrList:
        if arrItem.get(keyName) == keyValue:
            return arrItem
    return {} 

def mergeDicts(dicts):
    super_dict = {}
    for singleDict in dicts:
        for k, v in singleDict.items(): 
            super_dict[k] = v
    return super_dict

    
def getActivities(date):
    activitiyResponse = auth2_client.activities(date=date)

    activitySummary = activitiyResponse['summary'];
    activityData = {
        'Calories Burned':safeGet(activitySummary,'caloriesOut'),
        'Calories BMR': safeGet(activitySummary,'caloriesBMR'),
        'Resting Heart Rate':safeGet(activitySummary,'restingHeartRate'),
        'Steps':safeGet(activitySummary,'steps'),
        'Distance (Km)':filterArrObj(activitySummary.get('distances', []), 'activity', 'total').get('distance', np.nan),
        'Elevation (Ft)':activitySummary['elevation'],
        'Floors':activitySummary['floors'],
        'Minutes Sedentary':activitySummary['sedentaryMinutes'],
        'Minutes Lightly Active':activitySummary['lightlyActiveMinutes'],
        'Minutes Fairly Active':activitySummary['fairlyActiveMinutes'],
        'Minutes Very Active':activitySummary['veryActiveMinutes'],
        'Activity Calories': activitySummary['activityCalories'],
        'Active Score': activitySummary['activeScore'],
        'Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('minutes', np.nan),
        'Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('caloriesOut', np.nan),
        'Fat Burn minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('minutes', np.nan),
        'Fat Burn calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('caloriesOut', np.nan),
        'Peak minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('minutes', np.nan),
        'Peak calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('caloriesOut', np.nan),
        'Normal Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('minutes', np.nan),
        'Normal Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('caloriesOut', np.nan),
    }
    
    return activityData

def getSleep(date):
    sleepResponse = auth2_client_new.sleep(date=date)
    
    sleepData = {}
    for sleepLog in sleepResponse.get('sleep', []):
        if sleepLog.get('isMainSleep'):
            sleepLevelsSummary = sleepLog.get('levels', {}).get('summary', {})
            
            sleepData['Sleep Efficiency'] = safeGet(sleepLog, 'efficiency')
            sleepData['Minutes Asleep'] = safeGet(sleepLog, 'minutesAsleep')
            sleepData['Minutes to fall asleep'] = safeGet(sleepLog, 'minutesToFallAsleep')
            sleepData['Sleep Start time'] = safeGet(sleepLog, 'startTime')
            sleepData['Sleep End time'] = safeGet(sleepLog, 'endTime')
            
            sleepData['Time in bed'] = safeGet(sleepLog, 'timeInBed')
            
            sleepData['Minutes Deep sleep'] = safeGet(sleepLevelsSummary.get('deep', {}), 'minutes')
            sleepData['Deep sleep count'] = safeGet(sleepLevelsSummary.get('deep', {}), 'count')
            sleepData['% Deep sleep'] = percent(safeGet(sleepData, 'Minutes Deep sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Light sleep'] = safeGet(sleepLevelsSummary.get('light', {}), 'minutes')
            sleepData['Light sleep count'] = safeGet(sleepLevelsSummary.get('light', {}), 'count')
            sleepData['% Light sleep'] = percent(safeGet(sleepData, 'Minutes Light sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes REM sleep'] = safeGet(sleepLevelsSummary.get('rem', {}), 'minutes')
            sleepData['REM sleep count'] = safeGet(sleepLevelsSummary.get('rem', {}), 'count')
            sleepData['% REM sleep'] = percent(safeGet(sleepData, 'Minutes REM sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Asleep'] = sleepData['Minutes Deep sleep'] + sleepData['Minutes Light sleep'] + sleepData['Minutes REM sleep']
            sleepData['Minutes Awake'] = safeGet(sleepLevelsSummary.get('wake', {}), 'minutes')
            sleepData['Minutes Awake count'] = safeGet(sleepLevelsSummary.get('wake', {}), 'count')            
    return sleepData

def getDateData(date):
    weekDayNum = date.isoweekday()
    return {
        'Date': prettyPrintDate(date),
        'Day of Week': weekDayNum,
        'Is Weekday': weekDayNum<6,
        'Is Weekend': weekDayNum>5
    };

def fetchAllData(date):
    dateStr = prettyPrintDate(date)
    
    print("Fetching fitbit data for: " + dateStr)
    
    nextDate = date #+ timedelta(days=1) #Arul
    sleepData = getSleep(prettyPrintDate(nextDate))
    
    activitiesData = getActivities(dateStr)
    dateData = getDateData(date)
    
    mergedData = mergeDicts([dateData, activitiesData, sleepData])
    return mergedData

In [5]:
#getSleep('2018-08-21') #multiple entries
#getSleep('2018-07-22') #https://www.fitbit.com/sleep/2018-07-22/18934545047
#getSleep('2018-08-2') #no sleep data
#print(getActivities('2018-06-29'))
#print(getSleep('2018-08-28')) #no sleep data

#auth2_client.activities(date='2018-09-05')
#auth2_client_new.sleep(date='2018-09-04')
#fetchAllData(datetime.datetime.strptime('2018-09-04', DATE_FORMAT))

## Data fetch layer

In [11]:
database = connectAndLoadDb()

def shouldFetchDataForProvidedDate(providedDate, todaysDate, API_COUNTER):
    return (providedDate < todaysDate) and API_COUNTER < 100;

def fetchAndAppendToDb(date, database):    
    mergedData = fetchAllData(date)
    database = database.append(mergedData) #, ignore_index=True)
    return database;

def fetchData(database, refetchAll = False):
    data = []
    API_COUNTER = 0
    
    todaysDate = datetime.datetime.today()
    
    print("Date today is :" + prettyPrintDate(todaysDate))
    
    if refetchAll == True:
        lastEntryDate = datetime.datetime.strptime('2021-12-21', DATE_FORMAT)
    else:
        lastEntryDate = getLastEntryDate(database)
    
    print("Last entry in Db is of :" + prettyPrintDate(lastEntryDate))
    print("----------------------------------------------")
    
    dateToFetch = lastEntryDate + timedelta(days=1)

    while shouldFetchDataForProvidedDate(dateToFetch, todaysDate, API_COUNTER):    
        database = fetchAndAppendToDb(dateToFetch, data)
        dateToFetch = dateToFetch + timedelta(days=1)
        API_COUNTER = API_COUNTER+1
    
    print("----------------------------------------------")
    print("Data fill completed! 👍👍")
    return data

Connecting database...
Database connected!


In [12]:
#database = fetchData(database, refetchAll = True)
data = fetchData(database, refetchAll = True)
print(data)

Date today is :2022-01-28
Last entry in Db is of :2021-12-21
----------------------------------------------
Fetching fitbit data for: 2021-12-22
Fetching fitbit data for: 2021-12-23
Fetching fitbit data for: 2021-12-24
Fetching fitbit data for: 2021-12-25
Fetching fitbit data for: 2021-12-26
Fetching fitbit data for: 2021-12-27
Fetching fitbit data for: 2021-12-28
Fetching fitbit data for: 2021-12-29
Fetching fitbit data for: 2021-12-30
Fetching fitbit data for: 2021-12-31
Fetching fitbit data for: 2022-01-01
Fetching fitbit data for: 2022-01-02
Fetching fitbit data for: 2022-01-03
Fetching fitbit data for: 2022-01-04
Fetching fitbit data for: 2022-01-05
Fetching fitbit data for: 2022-01-06
Fetching fitbit data for: 2022-01-07
Fetching fitbit data for: 2022-01-08
Fetching fitbit data for: 2022-01-09
Fetching fitbit data for: 2022-01-10
Fetching fitbit data for: 2022-01-11
Fetching fitbit data for: 2022-01-12
Fetching fitbit data for: 2022-01-13
Fetching fitbit data for: 2022-01-14
Fetc

In [14]:
df = pd.DataFrame(data)

In [15]:
df.head()

,Date,Day of Week,Is Weekday,Is Weekend,Calories Burned,Calories BMR,Resting Heart Rate,Steps,Distance (Km),Elevation (Ft),...,Deep sleep count,% Deep sleep,Minutes Light sleep,Light sleep count,% Light sleep,Minutes REM sleep,REM sleep count,% REM sleep,Minutes Awake,Minutes Awake count
0,2021-12-22,3,True,False,1444,1172,62.0,4544,1.91,20,...,4.0,14.0,330.0,35.0,64.0,44.0,7.0,9.0,76.0,36.0
1,2021-12-23,4,True,False,1442,1172,62.0,4013,1.68,20,...,6.0,18.0,275.0,31.0,56.0,66.0,8.0,14.0,66.0,30.0
2,2021-12-24,5,True,False,1499,1172,62.0,4479,1.82,30,...,6.0,21.0,250.0,29.0,51.0,79.0,8.0,17.0,63.0,30.0
3,2021-12-25,6,False,True,1459,1172,62.0,3810,1.60,20,...,3.0,17.0,151.0,12.0,59.0,12.0,2.0,5.0,53.0,14.0
4,2021-12-26,7,False,True,1404,1172,63.0,3745,1.55,20,...,3.0,21.0,197.0,17.0,51.0,49.0,5.0,13.0,63.0,16.0


In [35]:
!pip install pymysql

In [38]:
!pip install sqlalchemy

In [16]:
import pymysql
import sqlalchemy
engine = sqlalchemy.create_engine("mysql+pymysql://" + DB_USER + ":" + DB_PASSWD + "@" + DB_HOST + "/" + 'fitbitdb')
df.to_sql(con=engine, name='fitbit_logs', if_exists='replace', index=False)

In [24]:

cursor.executemany("""
    INSERT INTO fitbit_logs ('Sleep Efficiency': 98, 'Minutes Asleep': 443, 'Minutes to fall asleep': 0, 'Sleep Start time': '2021-12-21T23:06:00.000', 'Sleep End time': '2021-12-22T07:45:30.000', 'Time in bed': 519, 'Minutes Deep sleep': 69, 'Deep sleep count': 4, '% Deep sleep': 14.0, 'Minutes Light sleep': 330, 'Light sleep count': 35, '% Light sleep': 64.0, 'Minutes REM sleep': 44, 'REM sleep count': 7, '% REM sleep': 9.0, 'Minutes Awake': 76, 'Minutes Awake count': 36, 'Calories Burned': 1444, 'Calories BMR': 1172, 'Resting Heart Rate': 62, 'Steps': 4544, 'Distance (Km)': 1.91, 'Elevation (Ft)': 20, 'Floors': 2, 'Minutes Sedentary': 741, 'Minutes Lightly Active': 84, 'Minutes Fairly Active': 30, 'Minutes Very Active': 2, 'Activity Calories': 335, 'Active Score': -1, 'Cardio minutes': 0, 'Cardio calories': 0, 'Fat Burn minutes': 39, 'Fat Burn calories': 107.2852, 'Peak minutes': 0, 'Peak calories': 0, 'Normal Cardio minutes': 1380, 'Normal Cardio calories': 1320.308, 'Day of Week': 3, 'Is Weekday': True, 'Is Weekend': False, 'Date': '2021-12-22')
    VALUES (%(name)s, %(gender)s)""", data)

#large_df = pd.concat(database, ignore_index=True)

AttributeError: 'list' object has no attribute 'cursor'

In [13]:
writeDbToExcelFile(database)

Writing database to filename: data/database_main.xls
Database updated with new entries!!


C:\Users\arulc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#database